<a href="https://colab.research.google.com/github/sivanandhini751/GitHub-1/blob/main/AESSESSMENT_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install pyspark --quiet


from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Corona_SQL_Assessment").getOrCreate()


from google.colab import files
uploaded = files.upload()


import zipfile, os
with zipfile.ZipFile("coronadataset.zip", "r") as zip_ref:
    zip_ref.extractall("/content/coronadataset")


region_df = spark.read.option("header", True).csv("/content/coronadataset/Region.csv", inferSchema=True)
case_df = spark.read.option("header", True).csv("/content/coronadataset/Case.csv", inferSchema=True)
timeprovince_df = spark.read.option("header", True).csv("/content/coronadataset/TimeProvince.csv", inferSchema=True)

print("✅ Datasets Loaded Successfully!")
region_df.show(5)
case_df.show(5)
timeprovince_df.show(5)


region_df.createOrReplaceTempView("region")
case_df.createOrReplaceTempView("cases")
timeprovince_df.createOrReplaceTempView("timeprovince")



print("\n🌍 Total Confirmed Cases by Province:")
spark.sql("""
SELECT province, SUM(confirmed) AS total_confirmed
FROM cases
GROUP BY province
ORDER BY total_confirmed DESC
""").show(10)

print("\n💀 Top 10 Provinces by Total Deaths:")
spark.sql("""
SELECT province, SUM(deceased) AS total_deaths
FROM cases
GROUP BY province
ORDER BY total_deaths DESC
LIMIT 10
""").show()

print("\n📅 Average Daily New Cases (from TimeProvince):")
spark.sql("""
SELECT province, ROUND(AVG(confirmed),2) AS avg_daily_confirmed
FROM timeprovince
GROUP BY province
ORDER BY avg_daily_confirmed DESC
""").show(10)

print("\n📊 Join Example — Add Population to Case Stats:")
spark.sql("""
SELECT c.province, r.population, SUM(c.confirmed) AS total_confirmed,
       ROUND(SUM(c.confirmed)/r.population*1000000,2) AS cases_per_million
FROM cases c
JOIN region r ON c.province = r.province
GROUP BY c.province, r.population
ORDER BY cases_per_million DESC
""").show(10)


from pyspark.sql.window import Window
import pyspark.sql.functions as F

window_spec = Window.partitionBy("province").orderBy("date")
case_with_running_total = case_df.withColumn(
    "running_total", F.sum("confirmed").over(window_spec)
)
print("\n🪟 Running Total Example (First 10 rows):")
case_with_running_total.show(10)


Saving coronadataset.zip to coronadataset (1).zip
✅ Datasets Loaded Successfully!
+-----------------+-------+----------+----------+
|         province|country|population|area_sq_km|
+-----------------+-------+----------+----------+
|   Andhra Pradesh|  India|  53903393|    162968|
|Arunachal Pradesh|  India|   1570458|     83743|
|            Assam|  India|  35607039|     78438|
|            Bihar|  India| 124799926|     94163|
|     Chhattisgarh|  India|  29436231|    135194|
+-----------------+-------+----------+----------+
only showing top 5 rows

+--------------+----------+---------+--------+--------+
|      province|      date|confirmed|released|deceased|
+--------------+----------+---------+--------+--------+
|Andhra Pradesh|2020-04-01|     1393|    2941|       9|
|Andhra Pradesh|2020-04-02|      275|    1541|      55|
|Andhra Pradesh|2020-04-03|     1047|    2316|       8|
|Andhra Pradesh|2020-04-04|      779|    1083|      75|
|Andhra Pradesh|2020-04-05|     1938|    1495|     